02.03.2020
###  Работа с данными 
## Семинар 1

**Задача 1**

2/3 трафика идет на прямые ссылки из поисковых систем, конверсия в просмотр 10%. Остальной трафик идет на ЦПБ, просмотров из него 30%. Найти общую конверию в просмотры.

Решение:

Событие $А$ - просмотр,
$H_1$ - прямой переход, $P(H_1) = 2/3$, $P(A|H_1) = 0.1$

$H_2$ - переход из ЦПБ, $P(H_2) = 1/3$, $P(A|H_2) = 0.3$

По формуле полной вероятности $P(A)= P(H_1)P(A|H_1)+P(H_2)P(A|H_2)$

In [11]:
pA = 2/3*0.1+1/3*0.3
print(f"Конверсия по обоим каналам равна {pA*100:.2f}%")

Конверсия по обоим каналам равна 16.67%


**Задача 2. Титаник** 

По данным колонок Class и Survived из датасета пассажиров Титаника найти вероятность выжить для пассажира 1 класса

$$
P(Survived = Yes | Class= 1st) = \frac{P(Class=1 | Survived = Yes) \cdot P(Survived = Yes)}{P(Class=1)}
$$

Решение:

В задачах с реальными данными вероятности заменяются частотами:

In [66]:
#загрузка датасета
import pandas as pd

df = pd.read_csv('Dataset.data', sep = ',', header=None, 
                 names=['Class','Age','Gender','Survived'], usecols = ['Class', 'Survived'])
df.head()

,Class,Survived
0,1st,yes
1,1st,yes
2,1st,yes
3,1st,yes
4,1st,yes


In [71]:
total_num = df.count()[0] #2201
pS = df[df.Survived == 'yes'].count()[0]/total_num #711

p1 = df[df.Class == '1st'].count()[0]/total_num #325

p1S = df[(df.Class == '1st') & (df.Survived=='yes')].count()[0]/df[df.Survived == 'yes'].count()[0] #203/711

pS1 = p1S*pS/p1
print(f"Вероятность выжить, если пассажир путешествовал первым классом {pS1*100:.2f} %")
print(f"Априорная вероятность выжить  {pS*100:.2f} %")

Вероятность выжить, если пассажир путешествовал первым классом 62.46 %
Априорная вероятность выжить  32.30 %


In [70]:
# а вообще можно было сделать сразу так

pS1a = df[(df.Class == '1st') & (df.Survived=='yes')].count()[0]/df[df.Class == '1st'].count()[0]


0.6246153846153846

Задача 3. Байесовский классификтор текстов

In [3]:
# считаем частоты по словам в спаме и не спаме - вычисление максимальной апостериорной вероятности
# 

## $c_{MAP} = arg max_{c from C} \frac{P(c)*P(x_1...x_n|c)}{P(X)}$

In [4]:
# строим векторы из текстовых данных
# для простоты предполагаем, что появление слов друг от друга не зависит

## $P(x_1...x_n|c)=\product P(x_i|c)$



 P(X) одинаково для всех классов, правдоподобие данных

In [ ]:
import pandas as pd

In [16]:
df = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['class','sms'])

In [17]:
df.head()

,class,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [18]:
df.count()

class    5572
sms      5572
dtype: int64

In [21]:
df['class']

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: class, Length: 5572, dtype: object

In [22]:
# тупой способ
num_objects, num_features = df.shape

In [23]:
num_objects, num_features

(5572, 2)

In [30]:
SPAM_CLASS = 'spam'

(df['class'] == SPAM_CLASS).sum()

747

In [31]:
df[df['class'] == SPAM_CLASS] # аналог where в sql

,class,sms
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...


In [42]:
spam_sms_num = (df['class'] == SPAM_CLASS).sum()

notspam_sms_num = (df['class'] != SPAM_CLASS).sum()


# априорные вероятности 
p_spam = spam_sms_num / num_objects

p_notspam = notspam_sms_num / num_objects

In [44]:
print(f'{p_spam:.4f}, {p_notspam:.4f}')

0.1341, 0.8659


In [38]:
test_word = 'Free'.lower()

In [72]:
# удаляем знаки препинания
import string

print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [73]:
sms_example = df['sms'].values[0]

In [74]:
sms_example = ''.join([char for char in sms_example if char not in string.punctuation])

In [75]:
sms_example

'Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat'

In [76]:
sms_example = ' '.join([word.lower() for word in sms_example.split()])

In [77]:
sms_example

'go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat'

In [78]:
#чуть менее тупой способ - оформление в функцию

def text_preprocess(sms_text: str):
    """Преобразоавние текста для анализа"""
    text_no_punctuation = ''.join([char for char in sms_text if char not in string.punctuation])
    
    text_lowercase = ' '.join([word.lower() for word in text_no_punctuation.split()])
    
    
    return text_lowercase

sms_example = df['sms'].values[0]



In [79]:
df = df.assign(processed_text = df['sms'].apply(text_preprocess))

In [80]:
df.head()

,class,sms,processed_text
0,ham,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i dont think he goes to usf he lives aroun...


In [ ]:
#вероятность встретить тестовое слово в спаме

df['processed_text'].apply(lambda row: test_word in row)

In [81]:
spam_test_word_entries = df[df['class']==SPAM_CLASS]['processed_text'].apply(lambda row: test_word in row).sum()

In [83]:
spam_test_word_entries/ spam_sms_num

0.26639892904953144